## $f(x, y, z) = 2x^2 + (3+0.1N)y^2+(4+0.1N)z^2+xy-yz+xz+x-2y+3z+N$

In [1]:
import numpy as np
import pandas as pd

In [2]:
#http://old.apmath.spbu.ru/ru/structure/depts/is/task2-2013.pdf
N = 1
eps = 10**(-6)

A = np.array([[4,1,1], [1,2*(3+0.1*N),-1], [1,-1,2*(4+0.1*N)]])
b = np.array([[1, -2, 3]]).T

fmin = 0.11226497595  # точное значение функции в точке минимума, посчитанное при помощи Wolfram Alpha

#def f(x, y, z): return 2*x**2 + (3+0.1*N)*y**2 + (4+0.1*N)*z**2 + x*y - y*z + x*z +x - 2*y + 3*z + N

def f(A, b, X):
    return float(0.5*np.matmul(np.matmul(X.T,A),X) + np.dot(X.T,b) + N)

def diff_f(A, b, X):
    return np.matmul(A,X) + b

def norm_vec(X):
    return np.sqrt(sum(X ** 2))

def step_gd(A, b, X): #gradient descent
    diff = diff_f(A, b, X)
    return float(-(norm_vec(diff)**2)/np.matmul(np.matmul(diff.T, A),diff))

def step_cd(A, b, X, e): #coordinate descent
    diff = diff_f(A, b, X)
    return float(-np.dot(e, diff)/np.matmul(np.matmul(e.T, A),e))

def distance(A, b, X):
    return float(norm_vec(diff_f(A, b, X)) / min(np.linalg.eigvals(A))) #(2.6)



In [3]:
#Метод наискорейшего градиентного спуска (МНГС)
print('Метод наискорейшего градиентного спуска (МНГС):')

X0 = np.array([[0, 0, 0]]).T #МНГС сходится для любого начального вектора
X1 = X0 + step_gd(A, b, X0)*diff_f(A, b, X0)

res = pd.DataFrame()
pd.set_option("display.precision", 10)
res = res.append({'x':float(X0[0]), 'y':float(X0[1]), 'z':float(X0[2]), 
                  'f(x,y,z)':float(f(A, b, X0)), 'step':float(step_gd(A, b, X0)), 
                  'distance to min':float(distance(A, b, X1))}, ignore_index=True) 
 
while distance(A, b, X1) >= eps:
    X0 = X1
    res = res.append({'x':float(X0[0]), 'y':float(X0[1]), 'z':float(X0[2]), 
                      'f(x,y,z)':float(f(A, b, X0)), 'step':float(step_gd(A, b, X0)),
                      'distance to min':float(distance(A, b, X0))}, ignore_index=True)        
      
    X1 += step_gd(A, b, X0)*diff_f(A, b, X0)
print(f'Результат работы алгоритма f({round(float(X1[0]),4)},{round(float(X1[0]),4)},{round(float(X1[0]),4)}) = {round(float(f(A, b, X0)),4)}')
res

Метод наискорейшего градиентного спуска (МНГС):
Результат работы алгоритма f(-0.2549,-0.2549,-0.2549) = 0.1123


,x,y,z,"f(x,y,z)",step,distance to min
0,0.0000000000,0.0000000000,0.0000000000,1.0000000000,-0.1200686106,0.1762689379
1,-0.1200686106,0.2401372213,-0.3602058319,0.1595197256,-0.2623074550,0.1762689379
2,-0.2249016072,0.3112256911,-0.2778691865,0.1160157182,-0.1292599912,0.0701569837
3,-0.2421901408,0.3134790981,-0.3018269277,0.1126196767,-0.2150940556,0.0161643029
4,-0.2514158705,0.3127892495,-0.2952342850,0.1123196849,-0.1407603370,0.0077900115
5,-0.2530897196,0.3151664437,-0.2973279181,0.1122740892,-0.2044933671,0.0026199412
6,-0.2542102711,0.3155201867,-0.2960303890,0.1122665966,-0.1417225083,0.0013267120
7,-0.2545856534,0.3157972152,-0.2964300964,0.1122652651,-0.2041610734,0.0004667985
8,-0.2547948175,0.3158406653,-0.2962035472,0.1122650276,-0.1417470845,0.0002368385
9,-0.2548597161,0.3158944081,-0.2962737730,0.1122649852,-0.2041532863,0.0000834339


In [4]:
#Метод наискорейшего покоординатного спуска (МНПС)
print('Метод наискорейшего покоординатного спуска (МНПС):')

X0 = np.array([[0, 0, 0]]).T #МНПС сходится для любого начального вектора
X1 = X0 + step_cd(A, b, X0, np.array(np.eye(3)[0]).T)*diff_f(A, b, X0)
e = np.array(np.eye(3)[0]).T #орт для первого шага

res = pd.DataFrame()
pd.set_option("display.precision", 10)
res = res.append({'x':float(X0[0]), 'y':float(X0[1]), 'z':float(X0[2]), 
                  'f(x,y,z)':float(f(A, b, X0)), 'step':float(step_gd(A, b, X0)), 
                  'distance to min':float(distance(A, b, X1))}, ignore_index=True)
 
while distance(A, b, X1) >= eps:
    for i in range(3):
        X0 = X1
        e = np.array(np.eye(3)[i]).T
        res = res.append({'x':float(X0[0]), 'y':float(X0[1]), 'z':float(X0[2]), 
                          'f(x,y,z)':float(f(A, b, X0)), 'unit vec': [int(i) for i in e], 'step':float(step_gd(A, b, X0)),
                          'distance to min':float(distance(A, b, X0))}, ignore_index=True)        
      
        X1 += step_gd(A, b, X0)*diff_f(A, b, X0)
print(f'Результат работы алгоритма f({round(float(X1[0]),4)},{round(float(X1[0]),4)},{round(float(X1[0]),4)}) = {round(float(f(A, b, X0)),4)}')
res

Метод наискорейшего покоординатного спуска (МНПС):
Результат работы алгоритма f(-0.2549,-0.2549,-0.2549) = 0.1123


,x,y,z,"f(x,y,z)",step,distance to min,unit vec
0,0.0000000000,0.0000000000,0.0000000000,1.0000000000,-0.1200686106,1.2924316696,NaN
1,-0.2500000000,0.5000000000,-0.7500000000,1.1437500000,-0.1154401388,1.2924316696,"[1, 0, 0]"
2,-0.2211399653,0.3152957779,-0.2997834587,0.1144568625,-0.2137956558,0.0414000449,"[0, 1, 0]"
3,-0.2491370319,0.3081384731,-0.3009251051,0.1125008683,-0.1935104520,0.0142148413,"[0, 0, 1]"
4,-0.2512008674,0.3154441494,-0.2961141707,0.1122921524,-0.1934035187,0.0047285476,"[1, 0, 0]"
5,-0.2540103454,0.3153151435,-0.2971235018,0.1122690697,-0.1609704548,0.0019665620,"[0, 1, 0]"
6,-0.2543564737,0.3156262920,-0.2961998206,0.1122657467,-0.1749870099,0.0008105481,"[0, 0, 1]"
7,-0.2547065486,0.3158491629,-0.2964060787,0.1122651331,-0.1603142525,0.0003822961,"[1, 0, 0]"
8,-0.2548054448,0.3158548797,-0.2962320484,0.1122650080,-0.1749685522,0.0001652593,"[0, 1, 0]"
9,-0.2548756163,0.3159026710,-0.2962734947,0.1122649825,-0.1603139261,0.0000779710,"[0, 0, 1]"
